# Topic Modeling - Actividad

Topic modeling es una técnica de aprendizaje automático no supervisado donde intentados descubrir tópicos que son abstractos al texto pero que pueden describir una colección de documentos. Es importante marcar que estos "tópicos" no son necesariamente equivalentes a la interpretación coloquial de tópicos, sino que responden a un patrón que emerge de las palabras que están en los documentos.

La suposición básica para Topic Modeling es que cada documento está representado por una mescla de tópicos, y cada tópico consite en una conlección de palabras.

## Direcciones
Intentaremos construir un pipeline de machine learning donde como entrada recibamos texto, ejecutemos todos los pasos que vimos en este notebook incluyendo:
 - Eliminación de stopwords
 - Tokenización
 - Stemming y Lemmatization
 - Procesamiento especico del tema
 - Creación de features utilizando algun metodo de reducción de dimensionalidad, SVD, LSI, LDA

, para luego utilizar estas features para entrenar un modelo que nos permita predecir alguna propiedad interesante del set de datos. En este caso en particular, donde estamos viendo tweets, algunos casos interesantes podrían ser:
 - Predecir el sector al que pertenece el tweet: Alimentación, Bebidas, etc.
 - Predecir el paso en el Marketing Funel al que pertece
 
En esta actividad les propongo realizar cambios en alguna de las etapas del procesamiento para modificar la performance del modelo resultante

<img src='https://github.com/santiagxf/M72109/blob/master/NLP/Docs/atap_0406.png?raw=1' />

## Preparación

### Instalamos las librerias necesarias

In [3]:
!pip install unidecode
!python -m spacy download es_core_news_sm

     |████████████████████████████████| 245kB 5.5MB/s eta 0:00:01
     |████████████████████████████████| 16.2MB 102.8MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp36-none-any.whl size=16172934 sha256=b9a8623aeb6d46d08dc215ceb11222b934b6596fba9ec0614d8786da694aa111
  Stored in directory: /tmp/pip-ephem-wheel-cache-_g_u4muj/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


### Set de datos

Descargamos el set de datos y lo cargmamos en un DataFrame

In [1]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv --directory-prefix ./Datasets/mascorpus/

--2020-09-01 13:05:20--  https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 512573 (501K) [text/plain]
Saving to: ‘./Datasets/mascorpus/tweets_marketing.csv’

tweets_marketing.cs 100%[===================>] 500.56K  --.-KB/s    in 0.03s   

2020-09-01 13:05:20 (16.4 MB/s) - ‘./Datasets/mascorpus/tweets_marketing.csv’ saved [512573/512573]



In [8]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets['TEXTO'], tweets['SECTOR'], 
                                                    test_size=0.33, 
                                                    stratify=tweets['SECTOR'])

## Construcción del modelo

### Pasos

**Paso 1:** Instanciamos nuestro preprocesamiento de texto

In [1]:
import unidecode
import spacy
import es_core_news_sm as spa
import re
import sklearn
import nltk
from nltk import stem
from nltk.corpus import stopwords
from nltk.tokenize.casual import TweetTokenizer

class TextNormalizer(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def __init__(self):
        nltk.download('stopwords', quiet=True)

        self.parser = spa.load() # Cargamos el parser en español
        self.tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True) # Creamos un tokenizer
        self.stemmer = stem.SnowballStemmer(language='spanish') # Creamos un steammer
        self.lemmatizer = lambda word : " ".join([token.lemma_ for token in self.parser(word)]) # Creamos un lemmatizer
        self.stopwords = set(stopwords.words('spanish')) # Instanciamos las stopwords en español
        self.urls_regex = re.compile('http\S+') # Usamos una expresion regular para encontrar las URLs
    
    def process_text(self, text):
        tokens = self.tokenizer.tokenize(text)
        tokens = [token for token in tokens if not re.match(self.urls_regex, token)]
        tokens = [token for token in tokens if len(token) > 4]
        tokens = [token for token in tokens if token not in self.stopwords]
        tokens = [unidecode.unidecode(token) for token in tokens] # Quitamos acentos
        tokens = [self.lemmatizer(token) for token in tokens]
        return tokens
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for doc in X:
            yield ' '.join(self.process_text(text=doc))

In [2]:
normalizer = TextNormalizer()

**Paso 2:** Instanciamos nuestro vectorizador, en este caso usando el método TF-IDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True, sublinear_tf=True, norm='l2')

**Paso 3:** Instanciamos nuestro generador de features

In [4]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

featurizer = LatentDirichletAllocation(n_components=7)

**Paso 4:** Instanciamos nuestro clasificador que utilizará las features generadas hasta este momento

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

estimator=GradientBoostingClassifier(learning_rate=0.01, 
                                     n_estimators=1500,
                                     max_depth=4, 
                                     min_samples_split=40, 
                                     min_samples_leaf=7, 
                                     subsample=1, 
                                     random_state=1234)

### Pipeline

Ensamblamos el pipeline

In [6]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline(steps=[('normalizer', normalizer), 
                           ('vectorizer', vectorizer),
                           ('featurizer', featurizer),
                           ('estimator', estimator)])

### Evaluación

**Evaluación:** Entrenamos el modelo y testeamos su performance

In [10]:
model = pipeline.fit(X=X_train, y=y_train)

In [28]:
predictions = model.predict(X_test)

In [30]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

ALIMENTACION       0.00      0.00      0.00       110
  AUTOMOCION       0.00      0.00      0.00       148
       BANCA       0.00      0.00      0.00       198
     BEBIDAS       0.28      0.27      0.27       223
    DEPORTES       0.30      0.53      0.39       216
      RETAIL       0.24      0.58      0.34       268
       TELCO       0.00      0.00      0.00        79

    accuracy                           0.27      1242
   macro avg       0.12      0.20      0.14      1242
weighted avg       0.15      0.27      0.19      1242



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Estos resultados son bastante probres como podemos ver. Intentemos buscar un poco de intuición en cuales son valores interesantes para un modelo de clasificación de este tipo utilizando Hyper-parameter tunning.

## Hyper-parameter tunning

Una forma más práctica de ver como nuestro modelo se comporta ante diferentes valores de los parametros es utilizando Hyper-parameter tunning. Este proceso busca configuraciones de parametros de los diferentes de nuestro modelo con el objetivo de mejorar una métrica que debemos especificar a optimizar en el proceso. Existen varias técnicas para optimización de parametros, desde técnicas sencillas como una busqueda exhaustiva donde se prueban todas las combinaciones, hasta búsquedas más avanzadas como Bayesian Optimization donde cada combinación de parametros que se prueba informa al proceso de generación sobre cuales son las direcciones a explorar en el espacio que son mas probables de tener buenos resultados.

### Random Search

En nuestro caso, utilizaremos una técnica relativamente sencilla llamada Random Search, o busqueda aleatoria. Consiste básicamente en muestrear diferentes valores de una distribución de parametros para así generar la siguiente configuración de parametros. A continuación definimos los espacios de busqueda para los paramteros:
 - n_components en LDA
 - n_estimators en nuestro GBT
 - learning_rate en nuestro GBT
 - max_depth en nuestro GBT
 
Noten como se especifican los parametros a optimizar y los rangos a incluir:

In [11]:
from sklearn.utils.fixes import loguniform

param_grid = {
    'featurizer__n_components': [10, 20, 30, 50, 100],
    'estimator__n_estimators' : [128, 512, 1024, 2048],
    'estimator__learning_rate':loguniform(1e-4, 1e-1),
    'estimator__max_depth':[2,4,8]
}

Luego, definimos cual es la métrica que queremos optimizar. En nuestro caso, al ser un problema de clasificación de multiples clases, una métrica interesante es el Average Precision (Weithed) lo cual es un promedio ponderado de las diferentes precisiones de cada clase. Podríamos estár tentados a utilizar accuracy, pero en esta configuración del problema no nos sería muy util.

In [12]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

scorer = sklearn.metrics.make_scorer(sklearn.metrics.precision_score, average = 'weighted')
tuning = RandomizedSearchCV(estimator = pipeline,
                            param_distributions = param_grid, 
                            scoring=scorer,
                            n_iter=10,
                            n_jobs=4,
                            cv=5)

En la configuración anterior generaremos 10 sets de parametros que se generaran a partir de muestras del espacio de búsqueda que indicamos. Para generar las métricas estamos especificando Cross Validation.

*Nota: La siguiente celda puede tardar bastante tiempo en ejecutarse. Colab podría abortar la sesión antes que termine*

In [13]:
tuning.fit(X=X_train, y=y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('normalizer', TextNormalizer()),
                                             ('vectorizer',
                                              TfidfVectorizer(sublinear_tf=True)),
                                             ('featurizer',
                                              LatentDirichletAllocation(n_components=7)),
                                             ('estimator',
                                              GradientBoostingClassifier(learning_rate=0.01,
                                                                         max_depth=4,
                                                                         min_samples_leaf=7,
                                                                         min_samples_split=40,
                                                                         n_estimators=1500,
                                                                         random_state

### Evaluación de los resultados

Podemos revisar los resultados de la búsqueda construyendo un DataFrame con los resultados de la siguiente forma:

In [29]:
data = pd.DataFrame(data=tuning.cv_results_['params'])
data['mean_test_score'] = tuning.cv_results_['mean_test_score']

Agrupamos y ordenamos los resultados para una visualización más sencilla:

In [58]:
grouped = data.groupby('featurizer__n_components').apply(lambda x:x.sort_values(by='mean_test_score'))

In [59]:
grouped.head(10)

estimator__learning_rate  estimator__max_depth  \
featurizer__n_components                                                     
10                       8                  0.000165                     2   
                         6                  0.003880                     8   
                         5                  0.000112                     8   
30                       4                  0.000301                     2   
                         7                  0.010742                     4   
                         9                  0.003124                     4   
                         1                  0.034349                     2   
                         0                  0.000571                     8   
                         3                  0.003226                     4   
100                      2                  0.023705                     4   

                            estimator__n_estimators  featurizer__n_components  \
featurizer__n_components                                                        
10                       8                      512                        10   
                         6                     1024                        10   
                         5                     2048                        10   
30                       4                     1024                        30   
                         7                      128                        30   
                         9                      512                        30   
                         1                     1024                        30   
                         0                     2048                        30   
                         3                     2048                        30   
100                      2                      128                       100   

                            mean_test_score  
featurizer__n_components                     
10                       8         0.172604  
                         6         0.333639  
                         5         0.341656  
30                       4         0.358786  
                         7         0.371221  
                         9         0.375723  
                         1         0.381000  
                         0         0.385264  
                         3         0.389510  
100                      2         0.548903

### Mejor clasificador

Obteniendo el mejor clasificador de la búsqueda que hicimos. El mismo corresponde a un GBT con:
 - learning_rate = 0.023705
 - max_depth = 4
 - n_estimators = 128

Nuestro generador de topicos es LDA con k = 100

In [60]:
best_pipeline = tuning.best_estimator_

Probamos el mismo con datos que nunca fueron vistos por el proceso de parameter tunning:

In [62]:
predictions = best_pipeline.predict(X_test)

In [63]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

ALIMENTACION       0.50      0.51      0.51       110
  AUTOMOCION       0.60      0.44      0.51       148
       BANCA       0.60      0.34      0.43       198
     BEBIDAS       0.55      0.74      0.63       223
    DEPORTES       0.59      0.53      0.56       216
      RETAIL       0.40      0.61      0.48       268
       TELCO       0.83      0.13      0.22        79

    accuracy                           0.52      1242
   macro avg       0.58      0.47      0.48      1242
weighted avg       0.55      0.52      0.50      1242

